# Introduction to CAOM

To enable future growth and enhance the ability of users to find data of interest to their science case, the CADC has developed the **Common Archive Observation Model (CAOM)**. CAOM (now at version 2.2) enables the storage of observational metadata from the complete set of telescopic data and searching through that metadata using a single interface. The generalized capability of CAOM comes at the expense of some model complexity and the requirement of adopting a language that is unfamiliar to users. To decrease the learning curve for users, the CADC exposes CAOM via a simplified search web page interface (AdvancedSearch), this same interface provides access to all observations stored in the archive. For users requiring access to more details of the observations and greater flexibility in query construction CAOM is also exposed via a Table Access Protocol (TAP) web service that can be accessed via tools like ‘Topcat’ or directly via any http aware client. CAOM greatly decreases the workload involved in maintaining an archive but still requires expert knowledge to ensure that the metadata is correctly translated from the observational record into the CAOM database system.

CAOM metadata records that describe a complete observation can be constructed using two pieces of software. Each can be used independently and users are not required to use both. Before beginning with creating a record, however, one should understand the structure of the metadata model.

CAOM consists of a database structure that describe the circumstances (Timing, Location and Wavelength/Frequency) of an observation, the nature observation (image, spectrograph, polarizing, receiver, etc.) and the processing level of different files associated with the observation (ie. the raw and calibrated data are stored in the same CAOM entry).

The top level of the model is the Observation. The Observation is the overall container for all associated datasets. Each associated data set is stored in a Plane, so an Observation is made up of multiple Planes data. Within each Plane are the actual data files that contain the observational data, the Artifacts of the Plane, a FITS file is a good example of an Artifact. Any complex data structures within an Artifact, such a Multi Extension FITS file, is further described by having each structure within the Artifact described as a Part. A single Artifact can contain multiple Parts. A Part within an Artifact is normally describable and findable via metadata stored within the Artifact (such as the headers of an MEF file). Sometimes, very very rarely, a Part might itself contain complex data structures that are not clearly separated by metadata within the file, each of the data structures (normally there is only one) within a Part is called a Chunk. Thus we have:

in
Observation

  ```-> Plane 
    -> Artifact
        -> Part 
            -> Chunk  
        -> Part 
            -> Chunk 
        -> Part 
            ...
  -> Plane
    -> Artifact 
        ... 
  -> Plane 
     ...```
     
A technical description of CAOM, including a UML, is available from [CAOM Reference Documentation](http://www.opencadc.org/caom2/)

To create a CAOM record for transfer into the CADC database search system can be achieved using the software too fits2caom2 or by developing your own software in Python base on the pyCAOM2 module or both! The best place to start is likely to examine an existing set of CAOM2 records as examples, likely ones that are for observations that are similar to those you are attempting to store within the CADC.




# Tutorial
This tutorial describes the mechanism for accessing ALMA metadata and creating corresponding CAOM2 metadata. The Python code below is dependent on the following packages all of which available in pypi and installable with the command `pip install <package>`:
1. `astropy` - [astropy](https://github.com/astropy/astropy)
2. `astroquery` - [astroquery](https://github.com/astropy/astroquery)
3. `caom2` - [caom2](https://github.com/opencadc/caom2tools)

## Step 1 - Model the data
One possible model is to map every Member ous id to a plane. **TBD: reasoning and possible discussion**

In [2]:
from astroquery.alma import Alma
results = Alma.query({'member_ous_id': 'uid://A001/X11a2/X11'}, science=False)
results.show_in_notebook()

idx,Project code,Source name,RA,Dec,Galactic longitude,Galactic latitude,Band,Spatial resolution,Frequency resolution,Array,Mosaic,Integration,Release date,Frequency support,Velocity resolution,Pol products,Observation date,PI name,SB name,Proposal authors,Line sensitivity (10 km/s),Continuum sensitivity,PWV,Group ous id,Member ous id,Asdm uid,Project title,Project type,Scan intent,Field of view,Largest angular scale,QA2 Status,COUNT,Science keyword,Scientific category,ASA_PROJECT_CODE
,,,deg,deg,deg,deg,,,kHz,,,s,,GHz,m / s,,,,,,,mJy/beam,mm,,,,,,,arcsec,arcsec,,,,,
0,2016.1.00330.S,AzTEC-3,150.08624999998295,2.5890300000001023,236.3744731264975,42.31636591440288,[6],1.3802496212413773,3906.25,12m,,1814.4,2,"[236.89..238.77GHz,3906.25kHz,849.4uJy/beam@10km/s,55.3uJy/beam@native, XX YY] U [238.77..240.64GHz,3906.25kHz,860.9uJy/beam@10km/s,56.2uJy/beam@native, XX YY] U [253.02..254.89GHz,3906.25kHz,877.2uJy/beam@10km/s,59uJy/beam@native, XX YY] U [254.89..256.77GHz,3906.25kHz,882.7uJy/beam@10km/s,59.5uJy/beam@native, XX YY]",4560.818977915666,XX YY,2017-03-25 01:40:24,"Riechers, Dominik",AzTEC-3_a_06_TM1,"Scoville, Nick; Pavesi, Riccardo; Smolcic, Vernesa; Karim, Alexander; carilli, Chris; Schinnerer, Eva; Capak, Peter;",0.8494474424707475,0.028704754143949133,2.8617756,uid://A001/X11a2/X10,uid://A001/X11a2/X11,uid://A002/Xbe5932/X1015,"Detailed Gas and Dust Physics in ""Normal"" Galaxies and Starbursts at z>5",S,TARGET,25.52365324481848,5.888078542529382,Y,0,"Lyman Break Galaxies (LBG), Sub-mm Galaxies (SMG)",Galaxy evolution,2016.1.00330.S
1,2016.1.00330.S,J0948+0022,147.23883369998777,0.37376649999999684,236.58959190120834,38.713308468389506,[6],1.3814147665271306,3906.25,12m,,181.44,2,"[236.89..238.77GHz,3906.25kHz,2.7mJy/beam@10km/s,173.3uJy/beam@native, XX YY] U [238.77..240.64GHz,3906.25kHz,2.7mJy/beam@10km/s,176.2uJy/beam@native, XX YY] U [253.02..254.89GHz,3906.25kHz,2.7mJy/beam@10km/s,184.8uJy/beam@native, XX YY] U [254.89..256.77GHz,3906.25kHz,2.8mJy/beam@10km/s,186.6uJy/beam@native, XX YY]",4560.818977915666,XX YY,2017-03-25 01:40:24,"Riechers, Dominik",AzTEC-3_a_06_TM1,"Scoville, Nick; Pavesi, Riccardo; Smolcic, Vernesa; Karim, Alexander; carilli, Chris; Schinnerer, Eva; Capak, Peter;",2.6636240090787386,0.0899750387324174,2.8617756,uid://A001/X11a2/X10,uid://A001/X11a2/X11,uid://A002/Xbe5932/X1015,"Detailed Gas and Dust Physics in ""Normal"" Galaxies and Starbursts at z>5",S,PHASE WVR,25.52365324481848,5.893048996315706,Y,0,"Lyman Break Galaxies (LBG), Sub-mm Galaxies (SMG)",Galaxy evolution,2016.1.00330.S
2,2016.1.00330.S,J1058+0133,164.62335416662103,1.5663400000005478,251.51067721245977,52.77395252779326,[6],1.5011423257561953,3906.25,12m,,302.4,2,"[236.89..238.77GHz,3906.25kHz,2.1mJy/beam@10km/s,134.8uJy/beam@native, XX YY] U [238.77..240.64GHz,3906.25kHz,2.1mJy/beam@10km/s,137.1uJy/beam@native, XX YY] U [253.02..254.89GHz,3906.25kHz,2.1mJy/beam@10km/s,143.8uJy/beam@native, XX YY] U [254.89..256.77GHz,3906.25kHz,2.2mJy/beam@10km/s,145.2uJy/beam@native, XX YY]",4560.818977915666,XX YY,2017-03-25 01:40:24,"Riechers, Dominik",AzTEC-3_a_06_TM1,"Scoville, Nick; Pavesi, Riccardo; Smolcic, Vernesa; Karim, Alexander; carilli, Chris; Schinnerer, Eva; Capak, Peter;",2.0723765383918473,0.07001741082844595,2.8617756,uid://A001/X11a2/X10,uid://A001/X11a2/X11,uid://A002/Xbe5932/X1015,"Detailed Gas and Dust Physics in ""Normal"" Galaxies and Starbursts at z>5",S,BANDPASS FLUX WVR,25.52365324481848,6.403801009282779,Y,0,"Lyman Break Galaxies (LBG), Sub-mm Galaxies (SMG)",Galaxy evolution,2016.1.00330.S


Now that we have the metadata, let's build a CAOM2 observation. There are 2 types of observations in CAOM2: simple and composite. Details...
This is a simple observation so let's build it. At a minimum an observation is part of a collection (ALMA in this case) and it has an unique ID. Note that the observation id must be compatible with the path part of a URI. We will use the member_ous_id in the ALMA metadata but transform it to be compatible.

**TODO** describe algorithm

In [3]:
obs_id = results[0]['Member ous id'].replace('uid://', '').replace('/', '_')
print('Observation ID: {}'.format(obs_id))
import caom2
obs = caom2.Observation('ALMA', obs_id, algorithm='exposure')

Observation ID: A001_X11a2_X11


Now fill in other Observation metadata:

**TODO** describe

In [4]:
# observation level metadata is common amongst all rows. Can retrieve
# from the first row
from datetime import datetime

fr = results[0]
obs.meta_release = datetime.strptime(fr['Observation date'].decode('ascii'), '%Y-%m-%d %H:%M:%S')
proposal = caom2.Proposal(fr['ASA_PROJECT_CODE'])
proposal.project = fr['Project code']
proposal.pi_name = fr['PI name']
proposal.title = fr['Project title']
proposal.keywords = set(fr['Science keyword'].split(','))
obs.proposal = proposal
instrument = caom2.Instrument('BAND {}'.format(fr['Band'][0]))
obs.instrument = instrument
obs.algorithm = caom2.Algorithm('Exposure')
obs.intent = caom2.ObservationIntentType.SCIENCE
obs.telescope = caom2.Telescope('ALMA-{}'.format(fr['Array'].decode('ascii')), 2225142.18, -5440307.37, -2481029.852)
obs.target = caom2.Target(name='TBD', target_type=caom2.TargetType.OBJECT)

print(obs)

Observation.collection : ALMA
Observation.observation_id : A001_X11a2_X11
Observation.algorithm : Algorithm.name : Exposure
Observation.sequence_number : None
Observation.intent : ObservationIntentType.SCIENCE
Observation.type : None
Observation.proposal : Proposal.id : 2016.1.00330.S
Proposal.pi_name : Riechers, Dominik
Proposal.project : 2016.1.00330.S
Proposal.title : Detailed Gas and Dust Physics in "Normal" Galaxies and Starbursts at z>5
Observation.telescope : Telescope.name : ALMA-12m
Telescope.geo_location_x : 2225142.18
Telescope.geo_location_y : -5440307.37
Telescope.geo_location_z : -2481029.852
Telescope.keywords : set()
Observation.instrument : Instrument.name : BAND 6
Observation.target : Target.name : TBD
Target.target_type : TargetType.OBJECT
Target.standard : None
Target.redshift : None
Target.keywords : set()
Target.moving : None
Observation.meta_release : 2017-03-25 01:40:24
Observation.planes : 
Observation.environment : None
Observation.target_position : None
Obser

## Plane

Time now to add the corresponding planes. Each plane is characterized by position, time, energy and polarization metadata. This is extracted from each of the rows in the `results` variable.

*Note:* Care should be taken using proper units when setting the values in the CAOM2 model. To do that, we use the `astropy.units` module to convert the units for each column into the units used by the CAOM2 model.

### Position
In ALMA's case the shape is circle with the center at (`RA`, `DEC`) and radius `Field of view/2`. Again, units are obtained from the table. CAOM2 units for `caom2.Point` defined by `RA` and `DEC` as well as `radius` are `degrees`. 

In [6]:
import astropy.units as u

def _get_position(row, table):
    # Extracts position from a returned row of the ALMA results table
    position = caom2.Position()
    position.resolution = row['Spatial resolution']
    # Shape is circle
    # make sure all units are degrees
    ra = row['RA'] * table['RA'].unit.to(u.deg)
    dec = row['Dec'] * table['Dec'].unit.to(u.deg)
    radius = row['Field of view'] * table['Field of view'].unit.to(u.deg) / 2.0
    circle = caom2.Circle(caom2.Point(ra, dec), radius)
    position.bounds = circle
    return position

# test
print(_get_position(results[0], results))

Position.bounds : Circle.center : Point.cval1 : 150.08624999998295
Point.cval2 : 2.5890300000001023
Circle.radius : 0.003544951839558122
Position.dimension : None
Position.resolution : 1.3802496212413773
Position.sample_size : None
Position.time_dependent : None


### Time

`Observation date` is the start date and `Integration` is the integration time. CAOM2 units are `second`.

**TODO** UTC transformation or not?

**TODO** bounds vs subinterval

In [21]:
from astropy.time import Time as AstropyTime

def _get_time(row, table):
    # Extracts time information from a rrow fo returned ALMA table
    time = caom2.Time()
    time.exposure = \
        row['Integration'] * table['Integration'].unit.to(u.second)
    time_lb = AstropyTime(datetime.strptime(
        row['Observation date'].decode('ascii'), '%Y-%m-%d %H:%M:%S'))
    time_ub = time_lb + time.exposure * u.second
    time_interval = caom2.Interval(time_lb.mjd, time_ub.mjd)
    samples = caom2.shape.SubInterval(time_lb.mjd, time_ub.mjd)
    time_interval.samples = [samples]
    time.bounds = time_interval
    return time

# test
print(_get_time(results[0], results))

Time.bounds : Interval.lower : 57837.06972222222
Interval.upper : 57837.09072222222
Interval.samples : [caom2.shape.SubInterval(lower=57837.06972222222,
                        upper=57837.09072222222)]
Time.dimension : None
Time.resolution : None
Time.sample_size : None
Time.exposure : 1814.4


### Energy

Energy bounds are extracted from the `Frequency support` field. The intervals in this field my overlap which is invalid in CAOM2. Therefore, the intervals are passed through the `_add_subinterval` function which resolves overlapping intervals.

**TODO** discusion on bounds vs. samples


In [22]:
import re

def _get_energy(row):
    def _add_subinterval(si_list, subinterval):
        if not si_list:
            return [subinterval]
        # check for overlaps
        # begining of the list?
        if subinterval[1] < si_list[0][0]:
            return [subinterval] + si_list
        if subinterval[0] > si_list[-1][1]:
            return si_list + [subinterval]
        result = []
        for si in si_list:
            if (si[0] >= subinterval[0] and si[0] <= subinterval[1]) or\
                (subinterval[0] >= si[0] and subinterval[0] <= si[1]):
                # overlap detected
                subinterval = (min(si[0], subinterval[0]),
                               max(si[1], subinterval[1]))
            else:
                if subinterval[0] < si[0]:
                    result += [subinterval]
                    result += si_list[si_list.index(si):]
                    return result
                else:
                    result += [si]
        return result + [subinterval]
    # Extracts the energy inform from a returned row
    min_bound = None
    max_bound = None
    si = None # list of non-overlapping sub-intervals
    for b in re.findall(r'\[([^]]*)\]', row['Frequency support']):
        e_int = b.split(',')[0]
        vals = e_int.split('..')
        lower_freq = float(vals[0])
        # upper string of form: 123.45GHz
        upper_str = re.findall(r'\b\d+\.?\d+', vals[1])[0]
        upper_freq = float(upper_str)
        units = u.Unit(vals[1][len(upper_str):])
        # wavelengths in meters:
        upper = (lower_freq*units).to(u.meter, equivalencies=u.spectral()).value
        lower = (upper_freq*units).to(u.meter, equivalencies=u.spectral()).value
        si = _add_subinterval(si, (lower, upper))
        if min_bound is not None:
            min_bound = min(min_bound, lower)
        else:
            min_bound = lower
        if max_bound is not None:
            max_bound = max(max_bound, upper)
        else:
            max_bound = upper
    samples = []
    for s in si:
        samples.append(caom2.shape.SubInterval(s[0], s[1]))
    bounds = caom2.Interval(min_bound, max_bound, samples=samples)
    return caom2.Energy(bounds=bounds)

# test
print(_get_energy(results[0]))

Energy.bounds : Interval.lower : 0.0011675525100284302
Interval.upper : 0.0012655344590316183
Interval.samples : [caom2.shape.SubInterval(lower=0.0011675525100284302,
                        upper=0.0011848567623112798), caom2.shape.SubInterval(lower=0.0012458130734707448,
                        upper=0.0012655344590316183)]
Energy.dimension : None
Energy.resolving_power : None
Energy.sample_size : None
Energy.bandpass_name : None
Energy.em_band : None
Energy.transition : None
Energy.restwav : None


### Polarization

The polarization information is extracted from the `Pol products` field.

In [23]:
def _get_polarization(row):
    # Extracts polarization information from a row
    polarization = caom2.Polarization()
    polarization.polarization_states = \
        [caom2.PolarizationState(i) for i in row['Pol products'].split()]
    return polarization

# test
print(_get_polarization(results[0]))

Polarization.dimension : None
Polarization.polarization_states : [<PolarizationState.XX: 'XX'>, <PolarizationState.YY: 'YY'>]


### Adding the planes to the observation

Now it's time to add all the planes to the created observation. The ID of the plane must be unique and be a valid path of a URI. In this case, we'll use the `Source name` and format it to be a valid path.

In [24]:
for row in results:
    productID = re.sub('-$', '', (
    re.sub('^-', '', ((re.sub('\W+', '-', row['Source name'])).
                      replace('--', '-')))))
    plane = caom2.Plane(productID)
    meta_release = \
        datetime.strptime(row['Observation date'].decode('ascii'),
                          '%Y-%m-%d %H:%M:%S')
    if 'Observation date' in row:
        meta_release = datetime.strptime(
            row['Observation date'].decode('ascii'), '%Y-%m-%d %H:%M:%S')
    plane.meta_release = meta_release
    tmp = row['Release date']
    if isinstance(tmp, bytes):
        tmp = tmp.decode('ascii')
    #TODO bug in astroquery.alma
    # plane.data_release = datetime.strptime(tmp, ALMA_RELEASE_DATE_FORMAT)

    plane.position = _get_position(row, results)
    plane.energy = _get_energy(row)
    plane.time = _get_time(row, results)
    plane.polarization = _get_polarization(row)

    plane.data_product_type = caom2.DataProductType.VISIBILITY
    plane.calibration_level = caom2.CalibrationLevel.CALIBRATED
    obs.planes[productID] = plane

print(obs)

Observation.collection : ALMA
Observation.observation_id : A001_X11a2_X11
Observation.algorithm : Algorithm.name : Exposure
Observation.sequence_number : None
Observation.intent : ObservationIntentType.SCIENCE
Observation.type : None
Observation.proposal : Proposal.id : 2016.1.00330.S
Proposal.pi_name : Riechers, Dominik
Proposal.project : 2016.1.00330.S
Proposal.title : Detailed Gas and Dust Physics in "Normal" Galaxies and Starbursts at z>5
Observation.telescope : Telescope.name : ALMA-12m
Telescope.geo_location_x : 2225142.18
Telescope.geo_location_y : -5440307.37
Telescope.geo_location_z : -2481029.852
Telescope.keywords : set()
Observation.instrument : Instrument.name : BAND 6
Observation.target : Target.name : TBD
Target.target_type : TargetType.OBJECT
Target.standard : None
Target.redshift : None
Target.keywords : set()
Target.moving : None
Observation.meta_release : 2017-03-25 01:40:24
Observation.planes : AzTEC-3 => Plane.product_id : AzTEC-3
Plane.creator_id : None
Plane.arti